In [28]:
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import os

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import warnings
warnings.filterwarnings('ignore')

In [66]:
# Determine important landmarks for plank
landmarks = [
    'NOSE',
    'LEFT_SHOULDER',
    'RIGHT_SHOULDER',
    'LEFT_ELBOW',
    'RIGHT_ELBOW',
    'LEFT_WRIST',
    'RIGHT_WRIST',
    'LEFT_HIP',
    'RIGHT_HIP',
    'LEFT_KNEE',
    'RIGHT_KNEE',
    'LEFT_ANKLE',
    'RIGHT_ANKLE',
    'LEFT_HEEL',
    'RIGHT_HEEL',
    'LEFT_FOOT_INDEX',
    'RIGHT_FOOT_INDEX',
]

headers = ['label'] 

# Generate all columns of the data frame
for landmark in landmarks:
    headers += [f'{landmark.lower()}_x', f'{landmark.lower()}_y', f'{landmark.lower()}_z', f'{landmark.lower()}_v']
print(f'Number of columns: {len(headers)}')

empty_df = pd.DataFrame(columns=headers)

Number of columns: 69


In [30]:
base_options = python.BaseOptions(model_asset_path='pose_landmarker.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

I0000 00:00:1718085116.699442 9861974 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718085116.824463 9915081 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718085116.934831 9915085 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [31]:
def create_image_data(files, features, label, df):
    df = df.copy()
    
    for file in files:
        image = mp.Image.create_from_file(file)

        pose_landmarks = detector.detect(image).pose_landmarks[0]
        # print('Pose landmarks:', pose_landmarks)
        # print('Number of Pose landmarks:', len(pose_landmarks)) # should be 33

        landmarks = []
        for feature in features:
            landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
            landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])
        landmarks = list(np.array(landmarks).flatten())
        # print('Number of landmarks:', len(landmarks))

        # Add the label to the landmarks
        sample = [label] + landmarks
        sample = pd.Series(sample, index=df.columns)

        # Add the sample to the data frame
        df = pd.concat([df, sample.to_frame().T], ignore_index=True)
    
    return df

In [51]:
def create_video_data(files, frame_capture_rate, features, label, df):
    df = df.copy()

    for file in files:
        frame_number = 0

        with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
            video = cv2.VideoCapture(file)
        
            while video.isOpened():
                ret, frame = video.read()
                
                if not ret:
                    break

                if frame_number % frame_capture_rate == 0:
                    landmarks = []
                    pose_landmarks = pose.process(frame).pose_landmarks.landmark

                    for feature in features:
                        landmark = pose_landmarks[mp.solutions.pose.PoseLandmark[feature].value]
                        landmarks.append([landmark.x, landmark.y, landmark.z, landmark.visibility])

                    landmarks = list(np.array(landmarks).flatten())
                    sample = [label] + landmarks

                    if len(sample) == len(df.columns):
                        sample = pd.Series(sample, index=df.columns)
                        df = pd.concat([df, sample.to_frame().T], ignore_index=True)
            
                frame_number += 1
            video.release()
       
    return df

# **Generate Data from Images**

In [6]:
image_data_path = 'data/plank/image'
video_data_path = 'data/plank/video'

In [12]:
correct_files = [os.path.join(image_data_path, 'correct', file) for file in os.listdir(os.path.join(image_data_path, 'correct'))]
print('Number of correct samples:', len(correct_files))

correct_df = create_image_data(correct_files, landmarks, 'c', empty_df)
print('Correct data frame shape:', correct_df.shape)
correct_df.head()

Number of correct samples: 12
Correct data frame shape: (12, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.106999,0.474287,0.041421,0.999992,0.204851,0.488668,-0.18609,0.999937,0.229784,...,0.165992,0.500833,0.893337,0.800755,-0.156181,0.965273,0.883106,0.752967,0.10702,0.515329
1,c,0.932826,0.475001,-0.000054,0.999962,0.799452,0.323116,0.251574,0.999886,0.813881,...,-0.084245,0.989843,0.120464,0.835478,0.194059,0.81017,0.086306,0.890535,-0.209508,0.990062
2,c,0.119034,0.49746,-0.114617,0.999986,0.243312,0.473431,-0.390952,0.999932,0.257345,...,0.427192,0.434707,0.928693,0.914889,-0.077621,0.873237,0.910039,0.877281,0.310827,0.421052
3,c,0.883873,0.498216,0.004287,0.999995,0.75265,0.383505,0.290709,0.99999,0.789969,...,-0.057218,0.997657,0.150336,0.819928,0.173781,0.956127,0.144818,0.86357,-0.159592,0.996758
4,c,0.916077,0.564887,0.018603,0.999997,0.781779,0.428076,0.255041,0.999986,0.810024,...,-0.055593,0.988392,0.137897,0.852291,0.140435,0.761842,0.123406,0.889846,-0.133799,0.988035


In [56]:
low_image_files = [os.path.join(image_data_path, 'low', file) for file in os.listdir(os.path.join(image_data_path, 'low'))]
print('Number of low samples:', len(low_image_files))
print(low_image_files)

low_image_df = create_image_data(low_image_files, landmarks, 'l', empty_df)
print('Low data frame shape:', low_image_df.shape)
low_image_df.head()

Number of low samples: 13
['data/plank/image/low/Screen Shot 2024-06-10 at 1.55.00 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.50.05 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 2.00.00 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.57.48 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.55.49 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.54.36 PM.png', 'data/plank/image/low/rawImage.jpg', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.41.08 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.55.35 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.48.12 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.57.30 PM.png', 'data/plank/image/low/Screen Shot 2024-06-10 at 1.57.16 PM.png', 'data/plank/image/low/plank-back-pain-form-mistake-1.jpg']
Low data frame shape: (13, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,l,0.101902,0.561759,-0.044028,0.999938,0.207304,0.377525,-0.207559,0.999781,0.22457,...,0.159017,0.306151,0.889678,0.822372,-0.158673,0.917595,0.855868,0.764802,0.103131,0.276856
1,l,0.880689,0.509248,-0.024418,0.999994,0.772448,0.237865,0.200576,0.999983,0.78795,...,-0.057278,0.997923,0.126143,0.771611,0.254396,0.965288,0.097072,0.862351,-0.1261,0.998315
2,l,0.89645,0.609938,-0.094008,0.999992,0.775156,0.448662,0.214646,0.999879,0.807127,...,-0.030357,0.950442,0.128417,0.896496,0.23014,0.555288,0.109461,0.925922,-0.06684,0.938204
3,l,0.906445,0.350456,-0.048803,0.999864,0.814106,0.119444,0.207929,0.999981,0.813866,...,-0.084347,0.992579,0.092681,0.66186,0.2286,0.903576,0.056413,0.804102,-0.232467,0.993057
4,l,0.065302,0.490127,-0.015589,0.999968,0.141306,0.273406,-0.271097,0.999978,0.204585,...,0.226206,0.474378,0.956583,0.86284,-0.222333,0.969115,0.951063,0.73628,0.147898,0.444942


In [55]:
high_image_files = [os.path.join(image_data_path, 'high', file) for file in os.listdir(os.path.join(image_data_path, 'high'))]
print('Number of high samples:', len(high_image_files))

high_image_df = create_image_data(high_image_files, landmarks, 'h', empty_df)
print('High data frame shape:', high_image_df.shape)
high_image_df.head()

Number of high samples: 13
High data frame shape: (13, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,h,0.879158,0.572394,-0.001965,0.999998,0.761738,0.473951,0.191882,0.999982,0.799332,...,-0.017896,0.994403,0.1704,0.7477,0.210187,0.778022,0.166921,0.784381,-0.087047,0.991719
1,h,0.163141,0.541749,-0.085495,0.999855,0.278051,0.442553,-0.23253,0.999546,0.273764,...,0.40498,0.035663,0.873996,0.69597,0.271223,0.288011,0.857829,0.689818,0.391697,0.029006
2,h,0.871076,0.666401,-0.006063,0.999997,0.789327,0.44269,0.237454,0.999969,0.808275,...,-0.138777,0.999456,0.196323,0.805368,0.214246,0.947438,0.12813,0.891945,-0.246104,0.999249
3,h,0.88381,0.660109,-0.001744,0.999996,0.789116,0.428211,0.242215,0.999969,0.811261,...,-0.097662,0.998134,0.193621,0.811241,0.236521,0.887194,0.140698,0.890875,-0.199125,0.997404
4,h,0.165611,0.608966,-0.060661,0.998918,0.246222,0.502463,-0.225676,0.999137,0.266306,...,0.205451,0.020024,0.894425,0.894556,-0.070551,0.284873,0.880512,0.84386,0.133038,0.011646


# **Generate Data from Videos**

In [54]:
correct_video_files = [os.path.join(video_data_path, 'correct', file) for file in os.listdir(os.path.join(video_data_path, 'correct'))]
print('Number of correct samples:', len(correct_video_files))

correct_video_df = create_video_data(correct_video_files, 5, landmarks, 'c', empty_df)
print('Correct video data frame shape:', correct_video_df.shape)
correct_video_df.head()


Number of correct samples: 1
data/plank/video/correct/correct_plank.mp4


I0000 00:00:1718086201.148667 9861974 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718086201.244074 9927238 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718086201.252286 9927238 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Correct video data frame shape: (96, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.903783,0.618764,0.043215,0.999855,0.801685,0.430241,0.296902,0.998821,0.827469,...,-0.154902,0.983322,0.143478,0.845839,0.05891,0.769544,0.110715,0.893111,-0.265165,0.983547
1,c,0.903815,0.621758,0.029574,0.999845,0.801964,0.430718,0.285536,0.998716,0.827322,...,-0.165709,0.982137,0.142661,0.846524,0.11971,0.762154,0.110673,0.893025,-0.274756,0.982493
2,c,0.904038,0.625424,0.02566,0.999832,0.802281,0.430987,0.28127,0.998599,0.827335,...,-0.174886,0.981043,0.143843,0.846265,0.225493,0.752609,0.111056,0.893074,-0.283028,0.981501
3,c,0.904319,0.629779,0.017256,0.999819,0.802523,0.431528,0.278283,0.998489,0.827503,...,-0.177487,0.980094,0.149597,0.846229,0.275183,0.742393,0.113696,0.893382,-0.285464,0.980542
4,c,0.904491,0.632291,0.01852,0.999811,0.802886,0.431876,0.277466,0.998405,0.827825,...,-0.191551,0.979468,0.153406,0.846235,0.29322,0.734431,0.115225,0.893912,-0.298465,0.979922


In [58]:
low_video_files = [os.path.join(video_data_path, 'low', file) for file in os.listdir(os.path.join(video_data_path, 'low'))]
print('Number of low samples:', len(low_video_files))

low_video_df = create_video_data(low_video_files, 5, landmarks, 'l', empty_df)
print('Low video data frame shape:', low_video_df.shape)
low_video_df.head()

Number of low samples: 1


I0000 00:00:1718086384.556991 9861974 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718086384.653705 9929050 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718086384.661130 9929051 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Low video data frame shape: (60, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,l,0.899706,0.70082,-0.061494,0.999762,0.787686,0.494229,0.217198,0.996827,0.810987,...,-0.133849,0.95664,0.093979,0.89564,0.155599,0.83682,0.056419,0.92323,-0.253256,0.970347
1,l,0.899292,0.6966,-0.047031,0.999746,0.789469,0.497304,0.222552,0.996742,0.810045,...,-0.122535,0.956436,0.108683,0.888164,0.15624,0.833154,0.062703,0.927005,-0.229811,0.969525
2,l,0.899142,0.69363,-0.038626,0.999729,0.790578,0.499034,0.227767,0.996631,0.809593,...,-0.122401,0.956208,0.113808,0.883391,0.163988,0.829616,0.065388,0.929349,-0.228902,0.968648
3,l,0.899106,0.691541,-0.034288,0.999715,0.791299,0.500437,0.231393,0.996528,0.809306,...,-0.123059,0.95614,0.116341,0.880546,0.165601,0.826418,0.066758,0.932137,-0.231444,0.967972
4,l,0.899093,0.690109,-0.033275,0.999703,0.79175,0.501808,0.23308,0.996431,0.809057,...,-0.123674,0.956119,0.117476,0.879561,0.168057,0.82403,0.067271,0.933863,-0.233337,0.967423


In [59]:
high_video_files = [os.path.join(video_data_path, 'high', file) for file in os.listdir(os.path.join(video_data_path, 'high'))]
print('Number of high samples:', len(high_video_files))

high_video_df = create_video_data(high_video_files, 5, landmarks, 'h', empty_df)
print('High video data frame shape:', high_video_df.shape)
high_video_df.head()

Number of high samples: 1


I0000 00:00:1718086410.083284 9861974 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
W0000 00:00:1718086410.179009 9929245 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718086410.187179 9929245 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


High video data frame shape: (78, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,h,0.895068,0.690565,0.018974,0.999848,0.800204,0.465185,0.30256,0.996815,0.818665,...,-0.18362,0.986964,0.154245,0.832613,0.298837,0.717325,0.107849,0.909471,-0.300584,0.987417
1,h,0.891302,0.689026,0.04269,0.999858,0.800204,0.461301,0.318602,0.997041,0.81865,...,-0.318171,0.986992,0.154246,0.835253,0.429672,0.72094,0.098629,0.913384,-0.429076,0.98757
2,h,0.889091,0.687866,0.042012,0.999867,0.800203,0.458147,0.323618,0.997231,0.818479,...,-0.314077,0.986967,0.154249,0.836796,0.417156,0.72368,0.098502,0.915606,-0.429323,0.987663
3,h,0.888622,0.68569,0.039346,0.999876,0.800506,0.454952,0.321663,0.997411,0.817896,...,-0.315712,0.987112,0.15436,0.838625,0.444398,0.729299,0.099032,0.919406,-0.43236,0.987972
4,h,0.887982,0.685301,0.03903,0.999885,0.800757,0.452756,0.320452,0.997572,0.817642,...,-0.313474,0.987308,0.153756,0.841629,0.433164,0.736667,0.09858,0.919289,-0.430448,0.988322


In [102]:
plank_data = pd.read_csv('data/plank/plank_data.csv')

plank_data['label'] = plank_data['label'].str.lower()

# Separate the classes
low_data= plank_data[plank_data['label'] == 'l']
high_data = plank_data[plank_data['label'] == 'h']
correct_data = plank_data[plank_data['label'] == 'c']

# Randomly sample 100 from each class

sample_size = 200
sample_low = low_data.sample(n=sample_size, random_state=42)
sample_high = high_data.sample(n=sample_size, random_state=42)
sample_correct = correct_data.sample(n=sample_size, random_state=42)

sampled_data = pd.concat([sample_low, sample_high, sample_correct])
print('Sampled data shape:', sampled_data.shape)
sampled_data.head()

Sampled data shape: (600, 69)


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
18240,l,0.873551,0.341211,0.006428,0.999818,0.754218,0.370173,0.241762,0.997989,0.780318,...,-0.127144,0.932999,0.096163,0.691377,0.083459,0.685030,0.057381,0.715348,-0.221229,0.922233
24266,l,0.186617,0.689645,-0.005351,0.999938,0.243449,0.715532,-0.143181,0.999678,0.253665,...,0.154528,0.811783,0.607543,0.870063,-0.113310,0.976028,0.594334,0.855911,0.114326,0.900786
8651,l,0.211332,0.493303,-0.176511,0.999972,0.287879,0.454376,-0.245247,0.999568,0.315570,...,0.264711,0.713180,0.903022,0.718910,0.002256,0.938012,0.870515,0.697409,0.245445,0.751438
23513,l,0.200117,0.723968,-0.014506,0.999910,0.255347,0.706452,-0.151369,0.999481,0.263588,...,0.129128,0.711834,0.621445,0.869514,-0.090647,0.945976,0.611136,0.857050,0.083203,0.783405
15179,l,0.084575,0.302032,-0.000133,0.999944,0.177219,0.293821,-0.198248,0.999584,0.208486,...,0.188373,0.785181,0.957764,0.624996,-0.182459,0.980558,0.918317,0.609237,0.105205,0.891345


# **Combine all data**

In [104]:
final_data = pd.concat([correct_df, low_image_df, high_image_df, correct_video_df, low_video_df, high_video_df, sampled_data], ignore_index=True)
print('Final data frame shape:', final_data.shape)
print('Unique labels:', final_data['label'].value_counts())
final_data.head()


Final data frame shape: (872, 69)
Unique labels: label
c    308
h    291
l    273
Name: count, dtype: int64


,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
0,c,0.106999,0.474287,0.041421,0.999992,0.204851,0.488668,-0.18609,0.999937,0.229784,...,0.165992,0.500833,0.893337,0.800755,-0.156181,0.965273,0.883106,0.752967,0.10702,0.515329
1,c,0.932826,0.475001,-0.000054,0.999962,0.799452,0.323116,0.251574,0.999886,0.813881,...,-0.084245,0.989843,0.120464,0.835478,0.194059,0.81017,0.086306,0.890535,-0.209508,0.990062
2,c,0.119034,0.49746,-0.114617,0.999986,0.243312,0.473431,-0.390952,0.999932,0.257345,...,0.427192,0.434707,0.928693,0.914889,-0.077621,0.873237,0.910039,0.877281,0.310827,0.421052
3,c,0.883873,0.498216,0.004287,0.999995,0.75265,0.383505,0.290709,0.99999,0.789969,...,-0.057218,0.997657,0.150336,0.819928,0.173781,0.956127,0.144818,0.86357,-0.159592,0.996758
4,c,0.916077,0.564887,0.018603,0.999997,0.781779,0.428076,0.255041,0.999986,0.810024,...,-0.055593,0.988392,0.137897,0.852291,0.140435,0.761842,0.123406,0.889846,-0.133799,0.988035


In [107]:
final_data.to_csv('data/plank/train_data.csv', index=False)